In [1]:
'''
#
# Obligatorisk karaktersatt oppgave #1
#
# Legg spesielt merke til at det er kun koden i klassen Kalman som kan endres. Det er koden som skal leveres inn
# Det er derfor viktig at INGEN ANNEN KODE ENDRES !!! 
#
'''
import pygame as pg
from random import random,randint
import numpy as np
from numpy.linalg import norm

fps = 0.0

class Projectile():

    def __init__(self, background, kalman=None):
        self.background = background
        self.rect = pg.Rect((800,700),(16,16))
        self.px = self.rect.x
        self.py = self.rect.y
        self.dx = 0.0
        self.kalm = kalman

    def move(self,goal):

        if self.kalm:
            goal = self.kalm.calc_next(goal)
        deltax = np.array(float(goal) - self.px)
        #print(delta2)
        mag_delta = norm(deltax)#* 500.0
        np.divide(deltax,mag_delta,deltax)

        self.dx += deltax
        #if self.dx:
            #self.dx /= norm(self.dx) * 50

        self.px += self.dx /50.0
        self.py += -0.5
        try:
            self.rect.x = int(self.px)
        except:
            pass
        try:
            self.rect.y = int(self.py)
        except:
            pass

pygame 2.0.0.dev12 (SDL 2.0.12, python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Target():

    def __init__(self, background, width):
        self.background = background
        self.rect = pg.Rect(self.background.get_width()//2-width//2, 
            50, width, 32)
        self.dx = 1 if random() > 0.5 else -1

    def move(self):
        self.rect.x += self.dx

        if self.rect.x < 300 or self.rect.x > self.background.get_width()-300:
            self.dx *= -1

    def noisy_x_pos(self):
        pos = self.rect.x
        center = self.rect.width//2
        noise = np.random.normal(0,1,1)[0]

        return pos + center + noise*300.0
#
# Her er Kalmanfilteret du skal utvikle
#



In [3]:
class Kalman():
    def __init__(self):
        self._dt = 1  # time step since it's just one I didn't use it in equations
        self._pos = 785 # postition estimated
        self._vel = 1 # velocity estimated
        self._pos_est_err = 300 # estimation error / uncertainty for postiion
        self._vel_est_err = 100 # estimation error / uncertainty for velocity 
        self._pos_mea_err = 300 # measurement error for position
        self._vel_mea_err = 100 # measurement error for velocity
        self._alpha = 1 # alpha
        self._beta = 1 # beta
        
        self._q = 27 # 27 process noise for position
        self._q_v = 10 # 10 process noise for velocity
        
    """ I have first tried using gamma, but that just makes things too complicated. Most of the time the Target has
    has constant velocity, it only accelerates once in each try and that's when changing position. So I thought there's no need
    to include acceleration or gamma, and just use a model where there's constant velocity. But since the velocity isn't
    constant 100% of the time in reality, we added some "process uncertainty" for velocity estimation error and position
    estimation error to make up for that, as described in KalmanFilter.net
    """


    def calc_next(self, zi):
        
        #Since dt = 1 we don't use self._dt in the code below as 
        #it doesn't make any difference in this case
        
        #State extrapolation
        self._pos = self._pos + self._vel
        
        #Covariance extrapolation
        self._pos_est_err = self._pos_est_err + self._vel_est_err + self._q
        self._vel_est_err = self._vel_est_err + self._q_v
        
        #Alhpa-beta update
        self._alpha = self._pos_est_err / (self._pos_est_err + self._pos_mea_err)
        self._beta = self._vel_est_err / (self._vel_est_err + self._vel_mea_err)
        
        #State update
        pos_prev = self._pos
        self._pos = pos_prev + self._alpha*(zi - pos_prev)
        self._vel = self._vel + self._beta*((zi - pos_prev)/self._dt)
        
        #Covariance update
        self._pos_est_err = (1-self._alpha)*self._pos_est_err
        self._vel_est_err = (1-self._beta)*self._vel_est_err

        return self._pos


In [4]:
pg.init()

w,h = 1600,800

background = pg.display.set_mode((w,h))
surf = pg.surfarray.pixels3d(background)
running = True
clock = pg.time.Clock()

kalman_score = 0
reg_score = 0
iters = 0
count = 0

while running:
    target = Target(background, 32)
    missile = Projectile(background)
    k_miss = Projectile(background,Kalman()) #kommenter inn denne linjen naar Kalman er implementert
    last_x_pos = target.noisy_x_pos
    noisy_draw = np.zeros((w,20))

    trial = True
    iters += 1

    while trial:

        # Setter en maksimal framerate på 300. Hvis dere vil øke denne er dette en mulig endring
        clock.tick(30000)
        fps = clock.get_fps()
        
        for e in pg.event.get():
            if e.type == pg.QUIT:
                running = False
        
        background.fill(0x448844)
        surf[:,0:20,0] = noisy_draw

        last_x_pos = target.noisy_x_pos()
        # print(last_x_pos)
        
        target.move()
        missile.move(last_x_pos)
        k_miss.move(last_x_pos) #kommenter inn denne linjen naar Kalman er implementert

        pg.draw.rect(background, (255, 200, 0), missile.rect)
        pg.draw.rect(background, (0, 200, 255), k_miss.rect) #kommenter inn denne linjen naar Kalman er implementert
        pg.draw.rect(background, (255, 200, 255), target.rect)

        noisy_draw[int(last_x_pos):int(last_x_pos)+20,:] = 255
        noisy_draw -= 1
        np.clip(noisy_draw, 0, 255, noisy_draw)

        coll = missile.rect.colliderect(target.rect)
        k_coll = k_miss.rect.colliderect(target.rect) #kommenter inn denne linjen naar Kalman er implementert#

        if coll:
            reg_score += 1

        if k_coll:    #kommenter inn denne linjen naar Kalman er implementert
            kalman_score += 1

        oob = missile.rect.y < 20

        if oob or coll or k_coll: #endre denne sjekken slik at k_coll ogsaa er med naar kalman er implementert
            trial = False

        pg.display.flip()

    print('kalman score: ', round(kalman_score/iters,2)) #kommenter inn denne linjen naar Kalman er implementert
    print('regular score: ', round(reg_score/iters,2))
    count += 1
    print("Nr. of tries:", count)

pg.quit()

kalman score:  1.0
regular score:  0.0
Nr. of tries: 1
kalman score:  1.0
regular score:  0.0
Nr. of tries: 2
kalman score:  1.0
regular score:  0.0
Nr. of tries: 3
kalman score:  0.75
regular score:  0.0
Nr. of tries: 4
kalman score:  0.8
regular score:  0.0
Nr. of tries: 5
kalman score:  0.67
regular score:  0.0
Nr. of tries: 6
kalman score:  0.71
regular score:  0.0
Nr. of tries: 7
kalman score:  0.75
regular score:  0.0
Nr. of tries: 8
kalman score:  0.78
regular score:  0.0
Nr. of tries: 9
kalman score:  0.7
regular score:  0.0
Nr. of tries: 10
kalman score:  0.73
regular score:  0.0
Nr. of tries: 11
kalman score:  0.75
regular score:  0.0
Nr. of tries: 12
kalman score:  0.77
regular score:  0.0
Nr. of tries: 13
kalman score:  0.79
regular score:  0.0
Nr. of tries: 14
kalman score:  0.73
regular score:  0.0
Nr. of tries: 15
kalman score:  0.69
regular score:  0.0
Nr. of tries: 16
kalman score:  0.71
regular score:  0.0
Nr. of tries: 17
kalman score:  0.67
regular score:  0.0
Nr. o

### 